# Discovery of Writing Differences - Huggingface Transformers

Capstone project by Tomo Umer

<img src="https://tomoumerdotcom.files.wordpress.com/2022/04/cropped-pho_logo_notext.png" style="width:400px;height:400px;"/>

## Imports

In [58]:
import pandas as pd
import numpy as np
import pickle

from datasets import Dataset
import evaluate
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback

from sklearn.preprocessing import LabelBinarizer

from sklearn.model_selection import train_test_split
from scipy import special


import plotly.express as px
from sklearn.metrics import confusion_matrix

## Setting up the Data

This next chunk was for using the 50 books per author. Results were ... ok.

In [59]:
# 03 is the one with limit 50 books per author
# library_select = pd.read_pickle('../data/library_select03.pkl')

# since hugging face only accepts up to 512 characters with this model,
# better to get words from the middle of the book
# library_select['book_content_modified'] = library_select['book_content'].apply(lambda text: text[len(text) // 2:])

In [60]:
# only 5 books per author
library_select = pd.read_pickle('../data/library_fixed_author_five.pkl')

In [61]:
select_authors = list(library_select['author'].unique())

In [62]:
# how many parts of a book to take
n_parts = 10 # previous was 5

bookpart_list = []

for i in range(n_parts):
    # note: the +1s are there because I don't want the exact beginning, or the end of the book (there could be some junk there)
    bookpart_list.append(library_select['book_content'].apply(lambda text: text[(i+1)* len(text) // (n_parts+1):]))

# copy the library n_parts times, to concatenate with the split texts
library_select_multi = pd.concat([library_select]*n_parts, ignore_index=True).drop(columns='book_content')

# add the above parts of the text into a new column
library_select_multi['book_part'] = pd.concat(bookpart_list, ignore_index=True)

In [63]:
library_select_multi.shape

(1050, 14)

In [7]:
# for testing
# library_select['book_length'] = library_select['book_content'].str.len()
#library_select_multi['book_part_length'] = library_select_multi['book_part'].str.len()

# and then to verify that it's the same book - 105 is of course with 21 authors, 5 books each
# library_select_multi.loc[0]
# library_select_multi.loc[105]
# library_select_multi.loc[210]

# library_select['author_num'].nunique()

In [64]:
checkpoint = 'bert-base-uncased'
model_path = '../models/bert_base_uncased'

In [9]:
# note truncation side and padding side are to determine which side to cutoff - beginning (left) or end (rigt)
tokenizer = AutoTokenizer.from_pretrained(checkpoint, truncation_side='right', padding_side='right')

def tokenize_function(df):
    return tokenizer(df['text'], truncation=True, padding='max_length',  max_length=512)

acc = evaluate.load('accuracy') #average = None
precision = evaluate.load('precision')
recall = evaluate.load('recall')
f1 = evaluate.load('f1')
mcc = evaluate.load('matthews_correlation')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc_m = acc.compute(predictions=predictions, references=labels)
    precision_m = precision.compute(predictions=predictions, average = 'macro', references=labels) #used  weighted for 50 books
    recall_m = recall.compute(predictions=predictions, average = 'macro', references=labels)
    f1_m = f1.compute(predictions=predictions, average = 'macro', references=labels)
    mcc_m = mcc.compute(predictions=predictions, references=labels)
    metrics = {
        'accuracy': acc_m['accuracy'],
        'precision': precision_m['precision'],
        'recall': recall_m['recall'],
        'f1': f1_m['f1'],
        'mcc': mcc_m['matthews_correlation']
    }
    return metrics

In [70]:
# # this part is the same if I use library_select ..
# select_authors = list(library_select_multi.sort_values(by='authorcentury')['author'].unique())

# authors_to_num = {select_authors[i]: i for i in range(len(select_authors))}
# num_to_authors = {v: k for k, v in authors_to_num.items()}

# library_select_multi['author_num'] = library_select_multi['author'].map(authors_to_num)
#authors_to_num

In [71]:
X = library_select_multi[['book_part']]
y = library_select_multi['author_num']

X_part, X_test, y_part, y_test = train_test_split(X, y, test_size=0.2, random_state = 42, stratify = y)

In [72]:
# to use for validation first
X_train, X_val, y_train, y_val = train_test_split(X_part, y_part, test_size=0.15, random_state = 42, stratify = y_part)

In [13]:
# split train further into train & 15% for validation (replace test here with validation)
train_ds = Dataset.from_dict({'text': X_train['book_part'], 'labels': y_train})
val_ds = Dataset.from_dict({'text': X_val['book_part'], 'labels': y_val})

In [14]:
tokenized_train_ds = train_ds.map(tokenize_function)
tokenized_train_ds = tokenized_train_ds.remove_columns(['text'])

Map:   0%|          | 0/714 [00:00<?, ? examples/s]

In [15]:
tokenized_val_ds = val_ds.map(tokenize_function)
tokenized_val_ds = tokenized_val_ds.remove_columns(['text'])

Map:   0%|          | 0/126 [00:00<?, ? examples/s]

In [16]:
# this needs to change if I change num authors
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=21)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [55]:
# this is only needed if running the trainer locally
training_args = TrainingArguments(
    output_dir=model_path,
    evaluation_strategy='epoch',
    num_train_epochs=15, #10 with last model
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    use_mps_device=True,
    save_total_limit=15, #10 with last model
    save_strategy='epoch',
    load_best_model_at_end=True,
    log_level ='info',
    metric_for_best_model='eval_mcc',
    optim = 'adamw_torch',
    learning_rate=1e-05,
    #fp16=True #this is to run on the gpu
)

Found safetensors installation, but --save_safetensors=False. Safetensors should be a preferred weights saving format due to security and performance reasons. If your model cannot be saved by safetensors please feel free to open an issue at https://github.com/huggingface/safetensors!
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [56]:
# this if uncommented to run the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_val_ds,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

In [57]:
# only run this on google collab, it takes 4 hours on my laptop!
trainer.train()

***** Running training *****
  Num examples = 714
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 675
  Number of trainable parameters = 109,498,389


  0%|          | 0/675 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 # only run this on google collab, it takes 4 hours on my laptop!                             │
│ ❱ 2 trainer.train()                                                                              │
│   3                                                                                              │
│                                                                                                  │
│ /Users/tomoumer/opt/anaconda3/envs/huggingface_ds6/lib/python3.11/site-packages/transformers/tra │
│ iner.py:1645 in train                                                                            │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /Users/tomoumer/opt/anaconda3/envs/huggingface_ds6/lib/python3.11/site-packages/transformers/tra │
│ iner.py:1943 in _inner_training_loop                                                             │
│                                                                                                  │
│   1940 │   │   │   │   if (                                                                      │
│   1941 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│   1942 │   │   │   │   │   and not is_torch_tpu_available()                                      │
│ ❱ 1943 │   │   │   │   │   and (torch.isnan(tr_loss_step) or torch.isinf(tr_loss_step))          │
│   1944 │   │   │   │   ):                                                                        │
│   1945 │   │   │   │   │   # if loss is nan or inf simply add the average of previous logged lo  │
│   1946 │   │   │   │   │   tr_loss += tr_loss / (1 + self.state.global_step - self._globalstep_  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

## Taking the pretrained model from google collab

In [73]:
#model = AutoModelForSequenceClassification.from_pretrained(model_path + '/checkpoint-58362')
model = AutoModelForSequenceClassification.from_pretrained('../models/bert_base_uncased/fivebooks_tenparts/')

loading configuration file ../models/bert_base_uncased/fivebooks_tenparts/config.json
Model config BertConfig {
  "_name_or_path": "../models/bert_base_uncased/fivebooks_tenparts/",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    

In [83]:
test_args = TrainingArguments(
    output_dir= '../models/bert_base_uncased/fivebooks_tenparts/',
    do_train=False,
    do_predict=True,
    use_mps_device=True,
    per_device_eval_batch_size=4,
    # fp16=True
)

Found safetensors installation, but --save_safetensors=False. Safetensors should be a preferred weights saving format due to security and performance reasons. If your model cannot be saved by safetensors please feel free to open an issue at https://github.com/huggingface/safetensors!
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [84]:
trainer = Trainer(
    model=model,
    args=test_args,
    compute_metrics=compute_metrics
)

In [76]:
test_ds = Dataset.from_dict({'text': X_test['book_part'], 'labels': y_test})

tokenized_test_ds = test_ds.map(tokenize_function)
tokenized_test_ds = tokenized_test_ds.remove_columns(['text'])

Map:   0%|          | 0/210 [00:00<?, ? examples/s]

In [85]:
results = trainer.predict(tokenized_test_ds) 

***** Running Prediction *****
  Num examples = 210
  Batch size = 4


  0%|          | 0/53 [00:00<?, ?it/s]

In [86]:
results.metrics

{'test_loss': 1.003623127937317,
 'test_accuracy': 0.8476190476190476,
 'test_precision': 0.8654880133871731,
 'test_recall': 0.8476190476190475,
 'test_f1': 0.8468330589368478,
 'test_mcc': 0.8411423259760178,
 'test_runtime': 422.9781,
 'test_samples_per_second': 0.496,
 'test_steps_per_second': 0.125}

In [87]:
results.predictions

array([[-0.46653876, -0.18904889, -1.4445355 , ...,  1.1178238 ,
         3.0759544 ,  0.6864331 ],
       [-1.2339908 ,  2.6362147 ,  0.8099453 , ...,  0.7417352 ,
        -0.31717333, -0.48127759],
       [ 0.20771548, -0.4811055 , -0.6417899 , ..., -0.8276385 ,
         1.4266744 ,  0.5450225 ],
       ...,
       [-1.1972811 ,  0.22819383, -0.6620481 , ...,  3.7477772 ,
         0.31357872,  0.5382277 ],
       [-1.1369017 ,  0.17825049, -0.64254266, ...,  3.7398863 ,
         0.04298076,  0.2895866 ],
       [ 0.12475164,  0.3906656 , -0.20289129, ..., -0.88842905,
         0.5512589 , -0.52083534]], dtype=float32)

In [88]:
# this is to get which labels are being predicted
results.predictions.argmax(axis=1)

# this simply stores the correct predictions, so equivalent to y_test:
# results.label_ids

array([19,  1, 13, 20, 15, 10,  9,  0,  2, 11,  0,  0, 18, 13, 17,  2,  5,
        9,  2,  8, 17,  3,  5,  1, 10, 19, 19,  3,  1, 16, 12,  6, 11,  0,
       19, 10, 11,  8, 19, 14,  1, 20,  1,  2,  9, 20,  4,  2, 12,  2,  9,
       12, 17,  8,  7,  6,  1, 19,  7,  2, 15,  8, 10, 14,  3,  6,  2, 12,
       14,  4, 13, 17, 13,  8,  5, 20, 14,  3, 11,  2, 15, 19, 18, 18, 15,
       15, 19,  4,  8,  5, 10, 18, 12,  5,  3, 18,  7, 15,  2, 17,  2, 17,
       10, 16,  7, 12, 16, 13,  6, 20, 20, 20, 20, 20,  4,  4, 11,  4, 15,
       10, 10, 15, 15, 10, 12, 15, 15, 17,  6,  5, 13, 17, 20, 13,  0, 19,
        9,  7, 14,  9,  7, 15,  6,  0, 16,  3,  4,  4,  0, 18,  2,  4, 12,
       14,  6, 12,  8, 16, 11, 18,  5,  6,  7,  6, 20, 11,  0,  2,  1,  3,
        8,  9,  0,  2,  9, 17,  2,  3,  1, 10,  1, 12, 11, 16,  3,  0, 19,
       16,  7,  6,  9, 11, 16, 10,  9, 17, 17,  3, 15, 18,  1, 18, 10,  3,
        2, 14,  2, 18, 18, 15])

In [89]:
select_authors = list(library_select['author'].unique())

In [121]:
# with open('../data/hf_testppred.pkl', 'wb') as f:
#      pickle.dump([y_test, results.predictions.argmax(axis=1), results.metrics], f)

In [91]:
fig = px.imshow(confusion_matrix(y_test, results.predictions.argmax(axis=1)),
                width=1000,
                height=800,
                text_auto=True,
                labels=dict(x='Predicted Label',
                            y='True Label'),
                            x=select_authors,
                            y=select_authors,
                            color_continuous_scale='Teal'
                            )

fig.update(layout_coloraxis_showscale=False)

fig.show()

## Introducing New Text

This part will ideally be in an app where any text can be uploaded

In [93]:
newtext = pd.DataFrame()

for book_num, book_name in enumerate(['Lambda', 'Deathway']):
        filepath = f'../data/{book_name} by Tomo Umer.txt'

        with open(filepath, encoding = 'utf-8') as fi:
                book = fi.read()
        
        tmp_text = pd.DataFrame({'id': f'TU{str(book_num).zfill(3)}',
                                 'title': [book_name],
                                 'author': 'Umer, Tomo',
                                 'authorcentury': 21,
                                 'book_content': [book]})

        newtext = pd.concat([newtext, tmp_text], ignore_index = True)

In [94]:
bookpart_list = []

for i in range(n_parts):
    # note: the +1s are there because I don't want the exact beginning, or the end of the book (there could be some junk there)
    bookpart_list.append(newtext['book_content'].apply(lambda text: text[(i+1)* len(text) // (n_parts+1):]))

# copy the library n_parts times, to concatenate with the split texts
newtext_multi = pd.concat([newtext]*n_parts, ignore_index=True).drop(columns='book_content')

# add the above parts of the text into a new column
newtext_multi['book_part'] = pd.concat(bookpart_list, ignore_index=True)

In [95]:
newtext_ds = Dataset.from_dict({'text': newtext_multi['book_part']}) #, 'labels': y_test})

tokenized_newtext_ds= newtext_ds.map(tokenize_function)
tokenized_newtext_ds = tokenized_newtext_ds.remove_columns(['text'])

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [96]:
new_results = trainer.predict(tokenized_newtext_ds) 

***** Running Prediction *****
  Num examples = 20
  Batch size = 4


  0%|          | 0/5 [00:00<?, ?it/s]

In [97]:
# first book Lambda, second Deathway
# newtext_multi['title'].unique()

# to see what got predicted
# new_results.predictions.argmax(axis=1)

# book1_authors = set(new_results.predictions.argmax(axis=1)[::2])
# book2_authors = set(new_results.predictions.argmax(axis=1)[1::2])

# for author in book1_authors:
#     print('Lambda is similar in writing to: ', num_to_authors[author])

# for author in book2_authors:
#     print('Deathway is similar in writing to: ', num_to_authors[author])

In [98]:
# this is needed for the strmlite app
authors_df = pd.read_pickle('../data/select_authors.pkl')

In [112]:
def compute_winners(model_predictions_max):
    unique_num, counts = np.unique(model_predictions_max, return_counts=True)

    unique_authors = [authors_df.loc[authors_df['author_num'] == unique, 'author'].iloc[0] for unique in unique_num]

    return pd.DataFrame({'most likely author': unique_authors, 'number of times':counts})

# Note that the above is because I put two books in at once. for one at a time, I would define this function
# to just give it predictions - like so
# def compute_winners(model_predictions):
#     unique_num, counts = np.unique(model_predictions.argmax(axis=1), return_counts=True)

#     unique_authors = [num_to_authors[unique] for unique in unique_num]

#     return pd.DataFrame({'most likely author': unique_authors, 'number of times':counts})

In [113]:
book1_winners = compute_winners(new_results.predictions.argmax(axis=1)[::2])
book2_winners = compute_winners(new_results.predictions.argmax(axis=1)[1::2])

In [114]:
book2_winners

,most likely author,number of times
0,"Dick, Philip K.",6
1,"Lovecraft, H. P. (Howard Phillips)",4


In [115]:
new_results.predictions.shape

(20, 21)

Note: the results are logits. Need to use softmax to convert them to probabilities.

In [116]:
#import torch.nn.functional as F
# new_probabilities = F.softmax(new_results, dim=-1)

In [117]:
new_probabilities = special.softmax(new_results.predictions, axis=1)

In [118]:
new_probabilities_df = pd.DataFrame(new_probabilities, columns=select_authors).T

In [119]:
new_probabilities_df[0].head().map('{:.2%}'.format)

Homer                     0.90%
Confucius                 2.16%
Plato                     0.95%
Cicero, Marcus Tullius    1.56%
Seneca, Lucius Annaeus    1.06%
Name: 0, dtype: object

In [120]:
for column in new_probabilities_df:
    print(new_probabilities_df[column].sort_values(ascending=False).head().map('{:.2%}'.format))

Dick, Philip K.                       55.24%
Doyle, Arthur Conan                   12.07%
Lovecraft, H. P. (Howard Phillips)     3.95%
Huxley, Aldous                         2.96%
Molière                                2.96%
Name: 0, dtype: object
Dick, Philip K.                       33.01%
Lovecraft, H. P. (Howard Phillips)    11.93%
Doyle, Arthur Conan                   10.78%
Molière                                7.92%
Huxley, Aldous                         4.58%
Name: 1, dtype: object
Dick, Philip K.                       37.43%
Doyle, Arthur Conan                   17.24%
Lovecraft, H. P. (Howard Phillips)     6.61%
Churchill, Winston                     5.55%
Huxley, Aldous                         4.70%
Name: 2, dtype: object
Dick, Philip K.                       55.30%
Doyle, Arthur Conan                    7.31%
Lovecraft, H. P. (Howard Phillips)     5.30%
Molière                                4.87%
Churchill, Winston                     2.93%
Name: 3, dtype: object
Dick, Ph

In [108]:
similar_books = pd.concat([newtext_multi ,pd.DataFrame(new_probabilities, columns=select_authors)], axis=1)

In [109]:
#similar_books.loc[similar_books['title'] == 'Lambda'].head()
similar_books.loc[similar_books['title'] == 'Deathway'].head()

,id,title,author,authorcentury,book_part,Homer,Confucius,Plato,"Cicero, Marcus Tullius","Seneca, Lucius Annaeus",...,"Defoe, Daniel","Jefferson, Thomas","Dickens, Charles","Doyle, Arthur Conan","Dumas, Alexandre","Twain, Mark","Churchill, Winston","Dick, Philip K.","Huxley, Aldous","Lovecraft, H. P. (Howard Phillips)"
1,TU001,Deathway,"Umer, Tomo",21,in the world left in her at that precise momen...,0.011767,0.033306,0.020630,0.009636,0.011020,...,0.014073,0.028561,0.024702,0.107751,0.014644,0.031148,0.035008,0.330138,0.045839,0.119293
3,TU001,Deathway,"Umer, Tomo",21,"rily lie down. “Just a quick break,” she told ...",0.009384,0.027308,0.013429,0.012603,0.008419,...,0.009009,0.020250,0.015575,0.073071,0.012141,0.026211,0.029301,0.552963,0.027216,0.053041
5,TU001,Deathway,"Umer, Tomo",21,"ing up with her laundry.\nAt the very least, h...",0.013791,0.028952,0.017620,0.015480,0.011818,...,0.014141,0.028479,0.041455,0.100375,0.023001,0.034004,0.037762,0.197822,0.136993,0.157733
7,TU001,Deathway,"Umer, Tomo",21,t to miss this party of yours.”\nThat last com...,0.015013,0.040081,0.027520,0.013259,0.010393,...,0.016293,0.033778,0.026798,0.157088,0.014020,0.036364,0.058297,0.237623,0.045027,0.107924
9,TU001,Deathway,"Umer, Tomo",21,ing mind games with her on this particular Thu...,0.016578,0.020951,0.021341,0.007261,0.010446,...,0.024458,0.033863,0.039557,0.044855,0.013498,0.040883,0.042861,0.083092,0.108967,0.346388


In [110]:
similar_books = (
    similar_books
        .drop(columns=['id', 'authorcentury', 'book_part', 'author'])
        .groupby('title')
        .mean()
        .pivot_table(columns='title')
)

In [111]:
print(similar_books.sort_values(by='Deathway', ascending=False)['Deathway'].head())
print('--------')
print(similar_books.sort_values(by='Lambda', ascending=False)['Lambda'].head())

Dick, Philip K.                       0.255924
Lovecraft, H. P. (Howard Phillips)    0.141731
Doyle, Arthur Conan                   0.106840
Huxley, Aldous                        0.063496
Molière                               0.054452
Name: Deathway, dtype: float32
--------
Dick, Philip K.                       0.381491
Doyle, Arthur Conan                   0.130887
Lovecraft, H. P. (Howard Phillips)    0.054233
Huxley, Aldous                        0.050893
Dumas, Alexandre                      0.041248
Name: Lambda, dtype: float32
